[HuggingFace(BERT)を使ってお手軽に日本語ニュースを分類する](https://qiita.com/ku_a_i/items/bac3befedbadab45774f)


In [2]:
import os
import pandas as pd

def get_title_list(path):
    """記事タイトル取得関数"""
    title_list = []
    filenames = os.listdir(path) #ファイル名称一覧取得
    for filename in filenames:
        # 1記事ずつファイルの読み込み
        with open(path+filename, encoding="utf_8_sig") as f:
            title = f.readlines()[2].strip() #各記事テキストの改行2番目に記事タイトルが記載してある
            title_list.append(title)
    return title_list

# データセットの生成(タイトルとラベル付与)
"""
今回は例として与えられた記事タイトルから
どのニュース媒体記事なのか？(独女通信、ITライフハック、MOVIE ENTERの3種類)
を分類する為のデータセットを作成する
"""
df = pd.DataFrame(columns=['label', 'sentence']) #空データフレーム

#独女通信(ラベル0)
title_list = get_title_list('./livedoor_news/dokujo-tsushin/')
for title in title_list:
    df = df.append({'label':0 , 'sentence':title}, ignore_index=True) #ignore_indexで合体後のindexを連番に

#ITライフハック(ラベル1)
title_list = get_title_list('./livedoor_news/it-life-hack/')
for title in title_list:
    df = df.append({'label':1 , 'sentence':title}, ignore_index=True)

#MOVIE ENTER(ラベル2)
title_list = get_title_list('./livedoor_news/movie-enter/')
for title in title_list:
    df = df.append({'label':2 , 'sentence':title}, ignore_index=True)

# 全データの順番をシャッフル(+index振り直し)
df = df.sample(frac=1 ,random_state=0).reset_index(drop=True)

#一応csvとしても保存しておく
df.to_csv('livedoor_sentence.csv', sep=',', index=False, encoding='utf_8_sig')

In [4]:
import collections

collections.Counter(df['label'])

Counter({1: 871, 2: 871, 0: 871})

In [6]:
from transformers import BertJapaneseTokenizer

"""
・BertJapaneseTokenizerは日本語用のBERTトークナイザ
・from_pretrainedで指定されたPytorchモデルの事前学習を実行する
・"cl-tohoku/bert-base-japanese-v2"は東北大学の日本語事前学習用BERTモデル ※下記補足参照
・do_subword_tokenizeは、サブワードのトークン化をするかどうか
・mecab_kwargsでMeCabユーザー辞書やNeoLogd等の指定も可能
　※"mecab_dic": Noneでデフォルト辞書(UniDic)をOFFにする必要あり
"""
tokenizer = BertJapaneseTokenizer.from_pretrained(
    "bert-base-japanese-v2", 
    #do_subword_tokenize=False,
    #mecab_kwargs={"mecab_dic": None, "mecab_option": "-d 'C:\mecab-unidic-neologd'"
)

In [7]:
#適当なキーワードでトークナイズしてみる
text = "楽しくリズム感覚が身につく"

#tokenizer.encodeでテキストをトークンIDに,return_tensors='pt'でPytorch型のテンソル型に変換
ids = tokenizer.encode(text, return_tensors='pt')[0]
wakati = tokenizer.convert_ids_to_tokens(ids) #どのようにトークナイズされたか分かち書きで確認
print(ids)
print(wakati)

tensor([    2, 32589, 17651, 16947,   862,  5128,   893, 12953,     3])
['[CLS]', '楽しく', 'リズム', '感覚', 'が', '身', 'に', 'つく', '[SEP]']


In [8]:
"""
・BertForSequenceClassificationはBEETの最終層をクラス分類に変えたもの
・事前学習はトークナイザと同じものを指定する
・num_labelsで分類数を指定する
"""
import torch
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-japanese-v2",
    num_labels = 3
)

#学習モードに設定
model.train()

#使用デバイス設定(CPU,GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device) #modelをGPUに転送

#オプティマイザの設定
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of the model checkpoint at bert-base-japanese-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

In [9]:
#乱数のseedを全固定する
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(0)

In [10]:
from torch.utils.data import TensorDataset, random_split, DataLoader, SequentialSampler, RandomSampler
from sklearn.model_selection import train_test_split

"""
・encodingで分類対象の文をトークナイザ
・input_idsはトークンID
・attention_mask はパディング用に埋め込み文字化どうかの判断用
・train_labels は分類ラベル
"""
encoding = tokenizer(df['sentence'].tolist(), return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
train_labels = torch.tensor(df['label'].tolist())

#データセット作成
dataset = TensorDataset(input_ids, attention_mask, train_labels)

#学習とテストの割合　※ここでは9割学習
train_size = int(0.9*len(dataset))
val_size= len(dataset) - train_size
train_dataset, val_dataset= random_split(dataset, [train_size, val_size])

print("学習データ数: {}" .format(train_size))
print("テストデータ数: {}" .format(val_size))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


学習データ数: 2351
テストデータ数: 262


In [11]:
import pandas as pd
import numpy as np

for i in range(val_dataset.__len__()):
    #別の方法あるかもだが、文字置換でスペシャルトークン(CLSとか)を消している
    sentence = tokenizer.decode(val_dataset.__getitem__(i)[0].detach().numpy().tolist()).replace('[CLS]', '').replace('[PAD]', '').replace('[SEP]', '').replace(' ', '')
    df_val_sentence_kari = pd.DataFrame(sentence.split(), columns={"sentence"})

    if i==0:
        df_val_sentence = df_val_sentence_kari.copy()
    else:
        df_val_sentence = pd.concat([df_val_sentence_kari, df_val_sentence], axis=0)

In [13]:
batch_size = 8

train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
#     sampler = RandomSampler(val_dataset), #先ほど元sentenceを保存した為シャッフルしない
    batch_size = 1 #テストなので1にした
)

In [14]:
def train(model):
    """学習ループ用関数"""
    model.train()
    train_loss = 0

    for batch in train_dataloader:
        b_input_ids = batch[0].to(device) #トークンID
        b_input_mask = batch[1].to(device) #埋め込み文字判断
        b_labels = batch[2].to(device) #正解ラベル
        optimizer.zero_grad()

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()    
    return train_loss

max_epoch = 10 #Epoch数は適当に
train_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    train_loss_.append(train_)

    print(train_)

98.95911388611421
26.422048819600604
10.551345311512705
4.299115178451757
2.3780125251214486
1.7822816642437829
1.591184270051599
0.9244664415746229
0.7345749257074203
0.7069113427023694


In [15]:
model.eval() #検証モード
preds_list = []
b_labels_list = []

for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device) #ラベルはモデルへ入力しないが正解確認用

    with torch.no_grad():
        preds = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        preds_list.append(preds) #予測ラベル
        b_labels_list.append(b_labels) #正解ラベル

#データフレームで結果を可視化する
for i in range(len(preds_list)):
    #preds_list[i][0]で一番確率が高いものを予測ラベルと判断(よくある分類のパターンと同じ)
    df_pred = pd.DataFrame(np.argmax(preds_list[i][0].cpu().numpy(), axis=1), columns={"pred_label"})
    df_label = pd.DataFrame(b_labels_list[i].cpu().numpy(), columns={"true_label"})
    df_result_kari = pd.concat([df_pred, df_label], axis=1)

    if i==0:
        df_result = df_result_kari.copy()
    else:
        df_result = pd.concat([df_result_kari, df_result], axis=0)

df_result = pd.concat([df_val_sentence, df_result], axis=1) #元タイトル(2-3で準備)と合体

In [16]:
len(df_result)

262

In [17]:
df_result.head(10)

,sentence,pred_label,true_label
0,癒し?それともドキドキ?女が美容室に求めるものとは,0,0
0,乃木坂46ではデジカメがブーム?「1000メディア出演キャラバン」でメンバーに聞く,1,1
0,アップルに対抗できるのか?マイクロソフトの独自タブレット【デジ通】,1,1
0,2ステップで録画開始!Youtubeやニコニコが録画できるB′s動画レコーダー2,1,1
0,ニコニコ超会議の来場者が9万人超え、配信での来場者は347万人超,1,1
0,【終了しました】原作者をも唸らせる仕上がりに期待!『るろうに剣心』舞台挨拶付きプレミア試写会...,2,2
0,【編集部的映画批評】水谷豊の“相棒”は座敷わらしだった,2,2
0,【週末映画まとめ読み】“美しすぎる”キャットウーマンが出演作を引っ提げ来日決定,2,2
0,男でも観たい!ヒットする少女コミック原作の映画,2,2
0,アイコンを変更してフォルダーを目立たせる【知っ得・虎の巻】,1,1


In [18]:
import csv

df_result.to_csv('livedoor_sentence_category.csv', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL) #encoding='sjis'だとJupyterLab上で表示不可なので注意